##### Grading Feedback Cell

# Question 0 (-2 If not answered)
Please provide the following the data so we can verify your github information and ensure accurate grading:
- Your Name:Abhijith Anil Vamadev
- Your SU ID: 495204994

# IST 718: Big Data Analytics

- Professors: 
  - Willard Williamson <wewillia@syr.edu>
  - Emory Creel <emcreel@g.syr.edu>
- Faculty Assistants: 
  - Warren Justin Fernandes <wjfernan@syr.edu>
  - Ruchita Hiteshkumar Harsora <	rharsora@g.syr.edu>

## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers from your classmates.  Short code snippets are allowed from the internet.  Code from the class text books or class provided code can be copied in its entirety.__
- Google Colab is the official class runtime environment so you should test your code on Colab before submission.
- Do not modify cells marked as grading cells or marked as do not modify.
- Before submitting your work, remember to check for run time errors with the following procedure:
`Runtime `$\rightarrow$ Factory reset runtime followed by Runtime $\rightarrow$ Run All.  All runtime errors will result in a minimum penalty of half off.
- All plots shall include descriptive title and axis labels.  Plot legends shall be included where possible.  Unless stated otherwise, plots can be made using any Python plotting package.
- Grading feedback cells are there for graders to provide feedback to students.  Don't change or remove grading feedback cells.
- Don't add or remove files from your git repo.
- Do not change file names in your repo.  This also means don't change the title of the ipython notebook.
- You are free to add additional code cells around the cells marked `your code here`.
- import * is not allowed because it is considered a very bad coding practice and in some cases can result in a significant delay (which slows down the grading process) in loading imports.  For example, the statement `from sympy import *` is not allowed.  You must import the specific packages that you need. 
- The graders reserve the right to deduct points for subjective things we see with your code.  For example, if we ask you to create a pandas data frame to display values from an investigation and you hard code the values, we will take points off for that.  This is only one of many different things we could find in reviewing your code.  In general, write your code like you are submitting it for a code peer review in industry.  
- Level of effort is part of our subjective grading.  For example, in cases where we ask for a more open ended investigation, some students put in significant effort and some students do the minimum possible to meet requirements.  In these cases, we may take points off for students who did not put in much effort as compared to students who put in a lot of effort.  We feel that the students who did a better job deserve a better grade.  We reserve the right to invoke level of effort grading at any time.
- Your notebook must run from start to finish without requiring manual input by the graders.  For example, do not mount your personal Google drive in your notebook as this will require graders to perform manual steps.  In short, your notebook should run from start to finish with no runtime errors and no need for graders to perform any manual steps.

In [12]:
%%bash
# Do not change or modify this cell
# Need to install pyspark
# if pyspark is already installed, will print a message indicating pyspark already installed
pip install pyspark >& /dev/null 

# Download the data files from github
# If the data file does not exist in the colab environment
data_file_1=US_Airline_Tweets.csv

if [[ ! -f ./${data_file_1} ]]; then 
   # download the data file from github and save it in this colab environment instance
   wget https://raw.githubusercontent.com/wewilli1/ist718_data/master/${data_file_1} >& /dev/null 
fi

# Sentiment Analysis
In this assignment, you will use a twitter US airline dataset to perform sentiment analysis.  Specifically, you will use twitter data to predict the sentiment of tweets related to peoples experience with an airline.

In [13]:
# Grading cell
# The purpose of the following boolean is to enable or disable grid search (see question 6a).  
# During grading we want to turn grid search off.  
# You should test your code with grid search set to False before submitting.
# Your notebook should run in its entirety without crashing when enable_grid is
# set to False before submitting.
enable_grid = False

##### Grading Feedback Cell

# Qustion 1: (10 pts)
Read US_Airline_Tweets.csv into a spark dataframe named `tweets_df`.  Drop all columns except airline_sentiment, airline, and text.  Drop rows in which the airline_sentiment column is labeled with a neutral sentiment.  Drop rows which contain NA / Null values in any column. Transform the airline_sentiment column such that a negative sentiment is equal to 0 and a positive sentiment is equal to 1.  This dataset has a lot more negative than positive tweets.  Balance the dataset such that the percentage of negative and positive tweets is roughly 50% each.  Your solution must **randomly sample** the dataset **without replacement** to perform balancing.  Determine and print the resulting percentage of positive and negative tweets in the dataframe such that it's easy for the graders to find and interpret your data.

In [17]:
# your code here
from pyspark.sql import SparkSession
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import Row
from pyspark.sql.functions import col,struct,when
from pyspark.sql import functions as fn
from pyspark.sql.types import StringType, StructType, IntegerType, StructField, BooleanType, DoubleType
from pyspark.sql.functions import col, sumDistinct, udf
from pyspark.sql.functions import col, explode, array, lit
from pyspark.ml.feature import Tokenizer, StopWordsRemover
#start spark
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
tweets_df = spark.read.format("csv").option("header","true").option("inferSchema","true").load("US_Airline_Tweets.csv") #reading all files
tweets_df = tweets_df.drop("tweet_id","airline_sentiment_confidence",
                           "negativereason","negativereason_confidence", 
                           "name", "airline_sentiment_gold",
                           "negativereason_gold", "retweet_count", 
                           "tweet_coord", "tweet_created", "tweet_location", 
                           "user_timezone")
tweets_df = tweets_df.filter(tweets_df.airline_sentiment != 'neutral')
tweets_df = tweets_df.na.drop("any")
def cre(inp):
  if(inp == 'positive'): # if the value of charges is less than or equal to the median returns 0 else 1
    return 1
  else:
    return 0


udf_func = udf(cre,IntegerType()) #creating user defined function
tweets_df = tweets_df.withColumn('airline_sentiment',udf_func(tweets_df.airline_sentiment).cast('int')) 

In [ ]:
print(tweets_df.filter(tweets_df.airline_sentiment == 1).count())
#minor_df = tweets_df.filter(tweets_df.airline_sentiment == 1)
#major_df = tweets_df.filter(tweets_df.airline_sentiment == 0)
#ratio = major_df.count()/minor_df.count()
#print("Before sampling, Positive vs Negative sentiment: {} vs {}".format(minor_df.count(), major_df.count()))

In [ ]:
tweets_df_sample = tweets_df.sampleBy('airline_sentiment',{0: 1/ratio, 1: 1})
mi_df = tweets_df.filter(tweets_df.airline_sentiment == 1)
ma_df = tweets_df.filter(tweets_df.airline_sentiment == 0)
tweets_df = tweets_df_sample


In [ ]:
print("After Sampling, Positive vs Negative sentiment: {} vs {}".format(tweets_df_sample.filter(tweets_df_sample.airline_sentiment == 1).count(),tweets_df_sample.filter(tweets_df_sample.airline_sentiment == 0).count()))


In [ ]:
# grading cell do not modify
tweets_pd = tweets_df.toPandas()
display(tweets_pd.head())
print(tweets_pd.shape)

##### Grading Feedback Cell

# Question 2: (10 pts)
Pre-process the data by creating a pipeline named `tweets_pre_proc_pipe`. Your pipeline should tokenize, remove stop words, and do a TF-IDF transformation.  Fit and execute your pipeline, and create a new dataframe named `tweets_pre_proc_df`.  Print the shape of the resulting TF-IDF data such that it's easy for the graders to find and understand as num rows x num words. Based on the shape of the TF-IDF data, would you expect a logistic regression model to overfit?

In [ ]:
# your code here
from pyspark.ml import Pipeline
import requests
from pyspark.ml.feature import RegexTokenizer, IDF,CountVectorizer, StopWordsRemover, HashingTF, HashingTF

stop_words = requests.get('http://ir.dcs.gla.ac.uk/resources/linguistic_utils/stop_words').text.split()

sw_filter = StopWordsRemover()\
  .setStopWords(stop_words)\
  .setCaseSensitive(False)\
  .setInputCol("words")\
  .setOutputCol("filtered")

cv = CountVectorizer()\
  .setInputCol("filtered")\
  .setOutputCol("tf")

tokenizer = RegexTokenizer().setGaps(False)\
  .setPattern("\\p{L}+")\
  .setInputCol("text")\
  .setOutputCol("words")

idf = IDF(inputCol="tf", outputCol="idf")

tweets_pre_proc_pipe =Pipeline(stages=[tokenizer, sw_filter, cv, idf])
tweets_fitted = tweets_pre_proc_pipe.fit(tweets_df)
tweets_pre_proc_df = tweets_fitted.transform(tweets_df)
print("Shape: {}, {}".format(tweets_pre_proc_df.count(), len(tweets_fitted.stages[-2].vocabulary)))

In [ ]:
# grading cell do not modify
tweets_pre_proc_df.show(10)

##### Grading Feedback Cell

Your explanation here: 

##### Grading Feedback Cell

# Question 3: (10 pts)
Since IDF considers a word's frequency across all documents in a corpus, you can use IDF as a form of inference.  Examine the documentation for the spark ML object that you used to create TF-IDF scores and learn how to extract the IDF scores for words in the corpus.  The idf object in your pipeline has a `values` attribute and a `tolist()` method which can be used to extract IDF values.  Create a pandas dataframe containing the 5 most important IDF scores named `most_imp_idf`.  Create another pandas dataframe containing the 5 least important IDF scores named `least_imp_idf`.  Each dataframe shall have 2 columns named `word` and `idf_score`.  Explain in words your interpretation of what the IDF scores mean.

In [ ]:
# your code here
from pyspark.sql.types import *
import pandas as pd
#dir(tweets_fitted.stages[-1])
#tweets_fitted.stages[-1].docFreq)
list_idf = tweets_fitted.stages[-1].idf.tolist()

low_value = pd.Series(list_idf[:5], name = "idf_score")
high_value = pd.Series(list_idf[-1:-6:-1], name = "idf_score")

vocab_list =  tweets_fitted.stages[-2].vocabulary
high_vocab =  pd.Series(vocab_list[:5], name = "Word")
low_vocab = pd.Series(vocab_list[-1:-6:-1], name = "Word")

most_imp_idf =pd.concat([high_vocab, high_value], axis=1)
least_imp_idf = pd.concat([low_vocab, low_value], axis = 1)

In [ ]:
# grading cell do not modify
display(most_imp_idf)
display(least_imp_idf)

##### Grading Feedback Cell

Your explanation here: 

##### Grading Feedback Cell

# Question 4: (10 pts)
Create a new recursive pipeline named `lr_pipe` which encapsulates `tweets_pre_proc_pipe` and adds a logistic regression model and any needed logistic regression support objects.  Use default logistic regression hyper parameters.  Fit lr_pipe using `tweets_df`.  Score the model using ROC AUC.  Report the resulting AUC such that it is easy for graders to find and interpret.

In [ ]:
# your code here
from pyspark.ml.classification import LogisticRegression

training_df, validation_df, testing_df = tweets_df.randomSplit([0.6, 0.3, 0.1], seed=0)
[training_df.count(), validation_df.count(), testing_df.count()]

lr = LogisticRegression().\
    setLabelCol('airline_sentiment').\
    setFeaturesCol('idf').\
    setRegParam(0.0).\
    setMaxIter(100).\
    setElasticNetParam(0.)
lr_pipe = Pipeline(stages=[tweets_pre_proc_pipe, lr]).fit(training_df)


In [ ]:
# see https://stackoverflow.com/questions/52847408/pyspark-extract-roc-curve
from pyspark.mllib.evaluation import BinaryClassificationMetrics
# Scala version implements .roc() and .pr()
# Python: https://spark.apache.org/docs/latest/api/python/_modules/pyspark/mllib/common.html
# Scala: https://spark.apache.org/docs/latest/api/java/org/apache/spark/mllib/evaluation/BinaryClassificationMetrics.html
class CurveMetrics(BinaryClassificationMetrics):
    def __init__(self, *args):
        super(CurveMetrics, self).__init__(*args)

    def _to_list(self, rdd):
        points = []
        # Note this collect could be inefficient for large datasets 
        # considering there may be one probability per datapoint (at most)
        # The Scala version takes a numBins parameter, 
        # but it doesn't seem possible to pass this from Python to Java
        for row in rdd.collect():
            # Results are returned as type scala.Tuple2, 
            # which doesn't appear to have a py4j mapping
            points += [(float(row._1()), float(row._2()))]
        return points

    def get_curve(self, method):
        rdd = getattr(self._java_model, method)().toJavaRDD()
        return self._to_list(rdd)

In [ ]:
# Create a Pipeline estimator and fit on train DF, predict on test DF

predictions = lr_pipe.transform(validation_df)
# Returns as a list (false positive rate, true positive rate)
preds = predictions.select('airline_sentiment','probability').rdd.map(lambda row: (float(row['probability'][1]), float(row['airline_sentiment'])))
points = CurveMetrics(preds).get_curve('roc')

plt.figure()
x_val = [x[0] for x in points]
y_val = [x[1] for x in points]
plt.title('IMDB Sentiment Analysis ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.plot(x_val, y_val)


In [ ]:
lr_pipe.transform(validation_df).select(fn.expr('float(prediction = airline_sentiment)').alias('correct')).\
    select(fn.avg('correct')).show()

##### Grading Feedback Cell

# Question 5: (10 pts)
Create 2 pandas dataframes named `lr_pipe_df_neg` and `lr_pipe_df_pos`which contain 2 colunms: `word` and `score`.  Load the 2 dataframes with the top 10 words and logistic regression coefficients that contribute the most to negative and positive sentiments respectively. Analyze the 2 dataframes and describe if the words make sense.  Do the words look like they are really negative and positive?

In [ ]:
# your code here
cv_words = pd.Series(lr_pipe.stages[-2].stages[-2].vocabulary, name = "Word")
scores = pd.Series(lr_pipe.stages[-1].coefficients, name = "Score")
words_score_df = pd.concat([cv_words, scores], axis=1)
words_score_df = words_score_df.sort_values(by=['Score'], ascending = False)
lr_pipe_df_neg = words_score_df.tail()
lr_pipe_df_pos = words_score_df.head()

In [ ]:
# grading cell - do not modify
display(lr_pipe_df_neg)
display(lr_pipe_df_pos)

##### Grading Feedback Cell

Your explanation here:

##### Grading Feedback Cell

# Question 6a: (5 pts)
The goal of this question is to try to improve the score from question 4 using an elastic net regularization grid search on a new pipeline named `lr_pipe_1`. lr_pipe_1 is the same as lr_pipe above but we would like you to create a new pipe for grading purposes only.  I'm not sure if it's possible to increase the score or not.  You will be graded on level of effort to increase the score in relation to other students in the class.  All of your grid search code should be inside the `if enable_grid` statement in the cell below.  The enable_grid boolean is set to true in a grading cell above.  If any of the grid search code executes outside of the if statement, you will not get full credit for the question.  We want the ability to turn off the grid search during grading.<br>

In [ ]:
# your grid search (and only your grid search) code here
from pyspark.ml.tuning import ParamGridBuilder

if enable_grid:
    # your grid search code here
    en_lr = LogisticRegression().\
        setLabelCol('airline_sentiment').\
        setFeaturesCol('idf')
    lr_pipe_1 = Pipeline(stages=[tweets_pre_proc_pipe, en_lr])
    lr_pipe_1_fitted = lr_pipe_1.fit(training_df)
    #lr_pipe_1.transform(validation_df).select(fn.avg(fn.expr('float(prediction = airline_sentiment)'))).show()
    grid = ParamGridBuilder().\
    addGrid(en_lr.regParam, [0., 0.01, 0.02, 0.004]).\
    addGrid(en_lr.elasticNetParam, [0., 0.2, 0.4, 0.8]).\
    addGrid(en_lr.maxIter, [0, 10, 50, 100, 300]).\
    addGrid(en_lr.threshold, [0, 0.5, 0.1, ]).\
    build()
    grid
    all_models = []
    for j in range(len(grid)):
        print("Fitting model {}".format(j+1))
        model = lr_pipe_1.fit(training_df, grid[j])
        all_models.append(model)
    pass


In [ ]:
# estimate the accuracy of each of them:
import numpy as np
accuracies = [m.\
    transform(validation_df).\
    select(fn.avg(fn.expr('float(airline_sentiment = prediction)')).alias('accuracy')).\
    first().\
    accuracy for m in all_models]
best_model_idx = np.argmax(accuracies)
print("best model index =", best_model_idx) 
grid[best_model_idx]
best_model = all_models[best_model_idx]
print(accuracies[best_model_idx])

In [ ]:
best_model_idx = np.argmax(accuracies)
print("best model index =", best_model_idx) 
grid[best_model_idx]
best_model = all_models[best_model_idx]
print(accuracies[best_model_idx])

In [ ]:
# estimate generalization performance
best_model.\
    transform(testing_df).\
    select(fn.avg(fn.expr('float(airline_sentiment = prediction)')).alias('accuracy')).\
    show()

##### Grading feedback cell

# Question 6b (5 pts)
Build a new pipeline named `lr_pipe_2` which uses the optimized model parameters from the grid search in question 6a above (the best model).  Create 2 variables named alpha and lambda and assign to them the best alpha and lambda produced by the grid search by hard coding the values. Fit and transform lr_pipe_2.  Compare AUC scores between lr_pipe_2 with lr_pipe in question 4.  Create a pandas dataframe named `comapre_1_df` which encapsulates the comparison data.  comapre_1_df Shall have 2 columns: `model_name` and `auc_score`.

In [582]:
# your optimized model code here
# example
# alpha = 0.1
# lambda = 0.1
elasticNetParam = best_model.stages[-1].getElasticNetParam()
regParam =best_model.stages[-1]. getRegParam()
maxIter = best_model.stages[-1].getMaxIter()
threshold = best_model.stages[-1].getThreshold()

lambda_par = 0.01
alpha_par = 0.4
en_lr_2 = LogisticRegression().\
        setLabelCol('airline_sentiment').\
        setFeaturesCol('idf').\
        setRegParam(lambda_par).\
        setMaxIter(maxIter).\
        setElasticNetParam(alpha_par).\
        setThreshold(threshold)
# lr_pipe_2 code here which uses the best alpha and lambda
lr_pipe_2 = Pipeline(stages=[tweets_pre_proc_pipe, en_lr_2])
lr_pipe_2_fitted = lr_pipe_2.fit(training_df)
lr_pipe_2_fitted.transform(validation_df)

DataFrame[airline_sentiment: int, airline: string, text: string, words: array<string>, filtered: array<string>, tf: vector, idf: vector, rawPrediction: vector, probability: vector, prediction: double]

In [22]:
# grading cell - do not modify
display(comapre_1_df)

NameError: ignored

##### Grading Feedback Cell

# Question 7 (10 pts)
Perform inference on lr_pipe_2.  Write code to report how many words were eliminated from the best model in question 6b above (if any) as compared to the model in question 4 above.  Make sure your output is easy for the graders to find and interpret.

Describe in words how feature selection is performed using elastic net regularization.

In [ ]:
# your code here

##### Grading Feedback Cell

Your explanation here:

##### Grading Feedback Cell

# Question 8 (10 pts)
Perform the same inference analysis that you did in question 5 but name the data frames `lr_pipe_df_neg_1` and `lr_pipe_df_pos_1`.  Compare the word importance results with the results in question 5.  Do the most positive and most negative words produced by using regularization better reflect positive and negative sentiment than the most positive and negative words produced by the model that did not use regularization?

In [ ]:
# your code here

In [ ]:
# grading cell - do not modify
display(lr_pipe_df_neg_1)
display(lr_pipe_df_pos_1)

##### Grading Feedback Cell

Your explanation here:

##### Grading Feedback Cell

# Question 9 (10 pts)
Precision recall plots are very similar to receiver operating characteristic (ROC) curves.  The high level steps for creating a precision recall curve are the same as the steps needed to create a ROC curve as outlined in lecture. Learn about [precision recall curves](https://en.wikipedia.org/wiki/Precision_and_recall).  Create a precision recall plot for the best model in question 6.  Describe what axes are the same / different between the precision recall curve and the ROC curve.

In [ ]:
# your code here

##### Grading Feedback Cell

Your explanation here:

##### Grading Feedback Cell

# Question 10 (10 pts)
Design and implement a method to rank the airlines in your dataset from best to worst.  Your solution can use model predictions or inference or both to perform this task. Implement your ranking algorithm in spark.  Create a spark dataframe named airline_rankings.  airline_rankings Shall have 3 columns: airline_name, num_reviews, and ranking.  Load the num_reviews column with the number of reviews associated with the airline.  Load the ranking column with your rank calculation result.  Sort airline rankings from best to worst (best at head, worst at tail).  

Describe in words how your algorithm works in clear easy to understand language.  We will take points off for descriptions that are not clearly stated and easy to follow.  We don't expect to have to reverse engineer your code understand how your algorithm works.

In [ ]:
# your code here

##### Grading Feedback Cell

Your algorithm description here:

In [ ]:
# grading cell do not modify
display(airline_rankings.toPandas().head())
display(airline_rankings.toPandas().tail())

##### Grading Feedback Cell

# Question 11 (0 pts)
Make sure to set enable_grid to False in the grading cell above and run the notebook in its entirety before submitting to verify that there are no runtime erros.  

##### Grading Feedback Cell